# Animal classification
https://www.kaggle.com/datasets/alessiocorrado99/animals10?resource=download

This dataset contains 10 classes of animals. TIt contains about 28K medium quality animal images belonging to 10 categories: dog, cat, horse, spyder, butterfly, chicken, sheep, cow, squirrel, elephant.

Each class has about 2k to 5k images.